In [37]:
import smtplib
from email.mime.text import MIMEText
import os
import google.genai as genai
from dotenv import load_dotenv
import json
load_dotenv()

True

In [4]:
sender_email = os.getenv('GOOGLE_MAIL_SENDER')
receiver_email = "arya.gokhale1@gmail.com"
password = os.getenv('GOOGLE_APP_PASSWORD')

In [ ]:
msg = MIMEText("This is a test email from Gmail SMTP.")
msg["Subject"] = "Hello from Gmail!"
msg["From"] = sender_email
msg["To"] = receiver_email

In [6]:
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, msg.as_string())

print("Email sent!")

Email sent!


In [8]:
def create_mail_prompt(mail_content, additional_info=None):
    prompt = f'''
    You are expert in writing emails.
    You go thoroughly through the content provided.
    The content is as follows:
    {mail_content}

    Think about a suitable subject for this email.
    Now think about perfect body that covers all the mail content.
    If mail content is any set of instructions/summary, provide the body in forms of bullets.
    If not, write a simple body that covers all aspects of the mail content provided.

    Always remember, no additional information from your side. No filler words and **highlighted headers**.
    Just return json as follows:
    subject:
    body:
    '''
    if additional_info is not None:
        prompt = prompt + '\n' + f"Here is some additional information about the content : {additional_info}"

    return prompt

In [32]:
def create_mail(mail_content, additional_info=None):
    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    prompt = create_mail_prompt(mail_content, additional_info)
    response = client.models.generate_content(
        model='gemini-2.0-flash-lite',
        contents=prompt
    )
    subject = client.models.generate_content(
        model='gemini-2.0-flash-lite',
        contents=[f'Extract only the subject from the following. No additional information please and no fillers please: {response.candidates[0].content.parts[0].text}']
    )
    body = client.models.generate_content(
        model='gemini-2.0-flash-lite',
        contents=[f'Extract only the body from the following. Add salutation. No additional information please and no fillers please: {response.candidates[0].content.parts[0].text}']
    )
    return {
        'subject': subject.candidates[0].content.parts[0].text,
        'body':body.candidates[0].content.parts[0].text
    }

In [33]:
content = '''
Scenario: Fraud Detection in Online Transactions

👩‍💻 Alice (Data Scientist): Hey team, we've been tasked with building a fraud detection system for online transactions. Our dataset contains transaction details like amount, location, device type, and whether the transaction was fraudulent. Where should we start?

👨‍💻 Bob (ML Engineer): First, we need to explore the dataset. Let’s check for missing values, outliers, and class imbalance. Fraudulent transactions are typically much fewer than non-fraudulent ones.

👩‍💻 Alice: Good point. If the dataset is highly imbalanced, we might need techniques like oversampling (SMOTE) or undersampling to balance the classes.

👨‍💻 Charlie (Business Analyst): Fraudulent transactions usually have patterns, like sudden high-value transactions from new devices or unusual locations. Feature engineering will be crucial.

👩‍💻 Alice: Exactly! We can create new features, such as transaction frequency per user, time since last transaction, and location anomalies.

👨‍💻 Bob: What model should we use? A simple Logistic Regression might work, but we could try Random Forest, XGBoost, or a Neural Network for better accuracy.

👩‍💻 Alice: XGBoost is great for tabular data. But let’s first build a baseline model with Logistic Regression and then optimize with XGBoost or LightGBM.

👨‍💻 Charlie: How do we evaluate the model? Accuracy might not be enough since fraud cases are rare.

👩‍💻 Alice: Right! Precision, Recall, and the F1-score are more important. We don’t want too many false negatives (missed fraud cases).

👨‍💻 Bob: Should we use a real-time detection system? If so, latency is a concern.

👩‍💻 Alice: Yes, if we deploy it, the model should be lightweight and fast, maybe using TensorFlow Lite or ONNX.

👨‍💻 Charlie: We should also ensure explainability. Banks and customers will want to know why a transaction was flagged as fraud.

👩‍💻 Alice: True! We can use SHAP values or LIME to explain model decisions.

👨‍💻 Bob: Let’s start with EDA, feature engineering, and a baseline model. Then, we can iterate and optimize.

👩‍💻 Alice: Sounds like a plan! Let’s get to work. 🚀
'''

In [34]:
mail_content = create_mail(content)

In [35]:
print(mail_content['subject'])
print(mail_content['body'])

Fraud Detection System

**Team,**

We're embarking on a project to build a fraud detection system for online transactions. Here’s a summary of our initial plan:

*   **Data Exploration:**
    *   Identify missing values, outliers, and class imbalance.
    *   Address class imbalance using techniques like oversampling (SMOTE) or undersampling.
*   **Feature Engineering:**
    *   Create new features based on patterns in fraudulent transactions.
    *   Examples: transaction frequency, time since last transaction, location anomalies.
*   **Model Selection:**
    *   Start with a baseline Logistic Regression model.
    *   Iterate and optimize using models like XGBoost or LightGBM.
*   **Model Evaluation:**
    *   Focus on Precision, Recall, and F1-score for evaluation.
    *   Minimize false negatives (missed fraud cases).
*   **Deployment Considerations:**
    *   If real-time, prioritize model speed and lightweight design (TensorFlow Lite, ONNX).
*   **Explainability:**
    *   Implem

In [ ]:
def send_mail(to, mail_content:json):
    msg =  MIMEText(mail_content['body'])
    subject = mail_content['subject']
    msg["Subject"] = subject
    msg["From"] = os.getenv('GOOGLE_MAIL_SENDER')
    msg["To"] = to
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_email, msg.as_string())
            print("Email sent successfully!")
    except Exception as e:
        print(f"Coudn't send email : {e}")
        return -1
    return 0

In [43]:
send_mail(to='arya.gokhale1@gmail.com', mail_content=mail_content)

Email sent successfully!


0

In [ ]:
# Mixing up!
def handle_mail(mail_content, to, additional_info=None):
    response_json = create_mail(mail_content, additional_info)
    send_mail(to, response_json)
    return 0

In [46]:
handle_mail(mail_content=content, to='arya.gokhale1@gmail.com', additional_info=None)

Email sent successfully!
